这里的代码将谱面文件转化为独热码形式的.npy文件

将音乐转化为频谱的代码在model_T_train.ipynb中

In [2]:
import os
import sys
import json
import librosa
import numpy as np
import re
from tqdm.notebook import tqdm
from math import ceil
import pickle
import matplotlib.pyplot as plt
import os
import shutil




class BpmError(Exception):
    pass
class MusicNotFound(Exception):
    pass

dic={
    (0, 0, 0, 0): 0, (0, 0, 0, 1): 1, (0, 0, 1, 0): 2, (0, 0, 1, 1): 3,
    (0, 1, 0, 0): 4, (0, 1, 0, 1): 5, (0, 1, 1, 0): 6, (0, 1, 1, 1): 7,
    (1, 0, 0, 0): 8, (1, 0, 0, 1): 9, (1, 0, 1, 0): 10, (1, 0, 1, 1): 11,
    (1, 1, 0, 0): 12, (1, 1, 0, 1): 13, (1, 1, 1, 0): 14, (1, 1, 1, 1): 15
}

In [3]:
def get_MC_map_label(path,map_name,step=int(1000/23)):#mc文件的处理
    global dic
    '''
        path:谱面文件路径
        map_name:谱面文件名称
        step:时间步，一秒钟多少个时间步
        
        为了简化操作把长条的开头算作一个单点
    '''
    

    #读取谱面文件
    try:
        with open(path+map_name,'r') as f:
            data=json.loads(f.read())
    except:
        with open(path+map_name,'r',encoding='utf-8') as f:
            data=json.loads(f.read())
            
    #获取bpm信息
    if len(data['time'])==1:
        bpm=round(data['time'][0]['bpm'],3)
    else:
        raise BpmError(f'谱面{path+map_name}含有多个bpm')

    #获取音乐名称
    music_name=data['note'][-1]['sound']
    
    for root,dirs,files in os.walk('labels'):
        if music_name+'.npy' in files:
            #print(music_name+'.npy   已存在')
            return 1
   
    try:
        #获取音乐相关信息
        y, sr = librosa.load(path+music_name, sr=None, duration=None)
        mu_len=librosa.get_duration(y=y, sr=sr)#音乐时长
        mu_step=round(mu_len*step)#音乐总的时间步数
        label=np.zeros((mu_step,4))
    except:
        raise MusicNotFound(f'[{path}]目录下找不到文件[{music_name}]')


    #获取偏移信息
    try:
        offset_step=data['note'][-1]['offset']/1000*step #谱面偏移占了多少个时间步
    except KeyError:#没有直接标明offset的情况下
        offset_beat=data['note'][-1]['beat']
        offset_step=(offset_beat[0]+offset_beat[1]/offset_beat[2])/bpm*60*step

    #将音符转化为labels
    for i in data['note'][:-1]:
        col=i['column']#谱面中的轨道，1234轨道分别对应0123
        if len(i)==2:#为单点音符时
            note_step=round((i['beat'][0]+i['beat'][1]/i['beat'][2])/bpm*60*step-offset_step)
            label[note_step,col]=1
            
        if len(i)==3:#为长条音符时---只计算开头为单点
            #ln_step:[step_start,step_end]
            ln_step=[round((i['beat'][0]+i['beat'][1]/i['beat'][2])/bpm*60*step-offset_step),
                     round((i['endbeat'][0]+i['endbeat'][1]/i['endbeat'][2])/bpm*60*step-offset_step)]
            label[ln_step[0],col]=1

#         if len(i)==3:#为长条音符时
#             #ln_step:[step_start,step_end]
#             ln_step=[round((i['beat'][0]+i['beat'][1]/i['beat'][2])/bpm*60*step-offset_step),
#                      round((i['endbeat'][0]+i['endbeat'][1]/i['endbeat'][2])/bpm*60*step-offset_step)]
#             label[ln_step[0]:ln_step[1],col,1]=1
#     np.save(path+music_name+'.npy', label)
    res=[]
    for i in label:
        temp=[0]*16
        res.append(dic[tuple(i)])
    res=np.array(res)
    #np.save(path+music_name+'.npy', res)
    return res,path+music_name

path='./0/朝你大胯捏一把 (Hardcore remix)/0/'
map_name='Hei Tuzi - Chao Ni Da Kua Nie Yi Ba(Hardcore remix) (XingRen) [Hard][key].mc'



In [4]:
def get_OSU_map_label(path,map_name,step=int(1000/23)):#OSU谱面文件的处理
    '''
        path:谱面文件路径
        map_name:谱面文件名称
        step:时间步，一秒钟多少个时间步
    '''
    global dic
    
    #读取谱面文件
    try:
        with open(path+map_name,'r') as f:
            data=f.read()
    except:
        with open(path+map_name,'r',encoding='utf-8') as f:
            data=f.read()
        
    #获取谱面的基础信息
    for row,i in enumerate(data.split('\n')):
        if 'AudioFilename' in i:
            music_name=i[15:]
            for root,dirs,files in os.walk('labels'):
                if music_name+'.npy' in files:
                    #print(music_name+'.npy   已存在')
                    return 1
            try:
                #获取音乐相关信息
                y, sr = librosa.load(path+music_name, sr=None, duration=None)
                mu_len=librosa.get_duration(y=y, sr=sr)#音乐时长
                mu_step=round(mu_len*step)#音乐总的时间步数
                label=np.zeros((mu_step,4))
            except:
                raise MusicNotFound(f'[{path}]目录下找不到文件[{music_name}]')
        #获取note开始的行数
        if 'HitObjects' in i:
            note_row=row+1
            break
    
    
    #获取note列参数信息
    col_set=set()
    for i in data.split('\n')[note_row:]:
        col_set.add(re.split(',|:',i)[0])
    col_set.remove('')
    col_list=sorted([int(i) for i in col_set])
    
    #将音符转化为labels
    for i in data.split('\n')[note_row:]:
        i=re.split(',|:',i)
        if len(i)<10:
            continue
        col=col_list.index(int(i[0]))
        if len(i)==10:#单点音符
            note_step=round(int(i[2])/1000*step)
            label[note_step,col]=1
        if len(i)==11:#长条音符--只计算开头为单点
            ln_step=[round(int(i[2])/1000*step),round(int(i[5])/1000*step)]
            label[ln_step[0],col]=1
#         if len(i)==11:#长条音符
#             ln_step=[round(int(i[2])/1000*step),round(int(i[5])/1000*step)]
#             label[ln_step[0]:ln_step[1],col,1]=1
#     np.save(path+music_name+'.npy', label)

    res=[]
    for i in label:
        res.append(dic[tuple(i)])
    res=np.array(res)
    #np.save(path+music_name+'.npy', res)
    return res,path+music_name
    
    
    
# path='./'
# map_name="Sharpnel Sounds - We Luv Lama (Night Bunny 7) [IcyWorld's Lv.29].osu"
# step=100
# get_OSU_map_label(path,map_name,step=100)

In [5]:
'''
osu的谱面和malody的offset不一样，这里只用malody的
'''
map_num=0
for root,dirs,files in os.walk('1'):
    for j in tqdm(files):
        if '.mc' in j:
            try:
                res,path=get_MC_map_label(root+'\\',j,step=22050/220)#->使得每个时间步都能对齐帧数，1秒中有多少个时间步。分母是22050，分子必须是整数。
                np.save(path+'[]'+str(map_num)+'.npy', res)
                map_num+=1
            except (BpmError,MusicNotFound) as e:
                print(e)
                pass
#                 print(root+'\\'+j)
#         if '.osu' in j:
#             try:
#                 res,path=get_OSU_map_label(root+'\\',j,step=22050/220)
#                 np.save(path+'[]'+str(map_num)+'.npy', res)
#                 map_num+=1
#             except (BpmError,MusicNotFound) as e:
#                 print(e)
#                 pass
#                 print(root+'\\'+j)
print(map_num)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

162
